In [ ]:
import numpy as np
import pandas as pd
from category_encoders import CatBoostEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn import model_selection
from sklearn.feature_selection import SelectFromModel

In [ ]:
train = pd.read_csv('../../Feature_Engineering/data/other-cleaned_train.csv')
test = pd.read_csv('../../Feature_Engineering/data/other-cleaned_test.csv')

In [ ]:
train.drop(columns = ['Unnamed: 0'], inplace = True)
test.drop(columns = ['Unnamed: 0'], inplace = True)

In [ ]:
X_train = train.copy()
X_test = test.copy()

In [ ]:
categ_columns = train.drop(columns = ["Opportunity_ID","ID", "Pricing, Delivery_Terms_Quote_Appr",\
                                    "Bureaucratic_Code_0_Approval","Bureaucratic_Code_0_Approved",\
                                    "Submitted_for_Approval","ASP","ASP_(converted)","TRF","Total_Amount",\
                                    "Total_Taxable_Amount","diferencia_en_dias","Last_Modified_DOY","Last_Modified_Year",\
                                    "Opportunity_Created_DOY","Opportunity_Created_Year","Quote_Expiry_DOY",\
                                     "Quote_Expiry_Year","Planned_Delivery_Start_DOY","Planned_Delivery_Start_Year",\
                                    "Planned_Delivery_End_DOY","Planned_Delivery_End_Year",\
                                    "Target"]).columns
for column in categ_columns:
    encoder = CatBoostEncoder()
    encoder.fit(train[column], train['Target'])
    feature_encoded = encoder.transform(train[column])
    X_train = X_train.join(feature_encoded.add_suffix('_cat_boost'))
    X_train.drop(columns=[column], inplace = True)

In [ ]:
categ_columns = test.drop(columns = ["Opportunity_ID","ID", "Pricing, Delivery_Terms_Quote_Appr",\
                                    "Bureaucratic_Code_0_Approval","Bureaucratic_Code_0_Approved",\
                                    "Submitted_for_Approval","ASP","ASP_(converted)","TRF","Total_Amount",\
                                    "Total_Taxable_Amount","diferencia_en_dias","Last_Modified_DOY","Last_Modified_Year",\
                                    "Opportunity_Created_DOY","Opportunity_Created_Year","Quote_Expiry_DOY",\
                                     "Quote_Expiry_Year","Planned_Delivery_Start_DOY","Planned_Delivery_Start_Year",\
                                    "Planned_Delivery_End_DOY","Planned_Delivery_End_Year"]).columns
for column in categ_columns:
    encoder = CatBoostEncoder()
    encoder.fit(train[column], train['Target'])
    feature_encoded = encoder.transform(test[column])
    X_test = X_test.join(feature_encoded.add_suffix('_cat_boost'))
    X_test.drop(columns=[column], inplace = True)

In [ ]:
X_train["Total_Amount"] = pd.to_numeric(X_train["Total_Amount"],errors='coerce').fillna(X_train["Total_Amount"].mean())
X_train["Opportunity_Created_Year"] = pd.to_numeric(X_train["Opportunity_Created_Year"],errors='coerce').fillna(0)
X_train["Quote_Expiry_DOY"] = pd.to_numeric(X_train["Quote_Expiry_DOY"],errors='coerce').fillna(0)
X_train["Quote_Expiry_Year"] = pd.to_numeric(X_train["Quote_Expiry_Year"],errors='coerce').fillna(0)
X_train["Planned_Delivery_End_DOY"] = pd.to_numeric(X_train["Planned_Delivery_End_DOY"],errors='coerce').fillna(0)
X_train["Planned_Delivery_End_Year"] = pd.to_numeric(X_train["Planned_Delivery_End_Year"],errors='coerce').fillna(0)

X_train = X_train.drop(columns = 'Target')

In [ ]:
X_test["Total_Amount"] = pd.to_numeric(X_test["Total_Amount"],errors='coerce').fillna(test["Total_Amount"].mean())
X_test["Opportunity_Created_Year"] = pd.to_numeric(X_test["Opportunity_Created_Year"],errors='coerce').fillna(0)
X_test["Quote_Expiry_DOY"] = pd.to_numeric(X_test["Quote_Expiry_DOY"],errors='coerce').fillna(0)
X_test["Quote_Expiry_Year"] = pd.to_numeric(X_test["Quote_Expiry_Year"],errors='coerce').fillna(0)
X_test["Planned_Delivery_End_DOY"] = pd.to_numeric(X_test["Planned_Delivery_End_DOY"],errors='coerce').fillna(0)
X_test["Planned_Delivery_End_Year"] = pd.to_numeric(X_test["Planned_Delivery_End_Year"],errors='coerce').fillna(0)

In [ ]:
def cross_val(model, x_train, y_train):
    score_cross_val = model_selection.cross_val_score(model, x_train, y_train, cv=5)
    print(score_cross_val.mean())

In [ ]:
def nusvc(x_train, y_train, x_validation, y_validation):
    nusvc = NuSVC(kernel='linear',probability=True)
    grid_nusvc = GridSearchCV(nusvc, param_grid={'nu': [0.4, 0.5]}, cv=5)
    grid_nusvc.fit(x_train, y_train)
    grid_best = grid_nusvc.best_estimator_
    print(grid_nusvc.best_params_)
    print('nusvc score: {}'.format(grid_best.score(x_validation, y_validation)))
    return grid_best

In [ ]:
def test_model(model, x_test, y_test):
    predictions = model.predict_proba(x_test)[:,1]
    logloss = log_loss(y_test, predictions)
    accuracy = accuracy_score(y_test, predictions.round())
    print("Accuracy: %.2f%%, Logloss: %.2f" % (accuracy*100.0, logloss))

In [ ]:
def best_features(model,train):
    importance = model.coef_[0]
    result = pd.DataFrame([train.columns,importance]).transpose()
    result.columns = ["Feature","Importance"]
    return result.sort_values(by='Importance', ascending=False).head(15)["Feature"]

In [ ]:
y = train.Target
x_train, x_validation, y_train, y_validation = train_test_split(X_train, y, test_size=0.2, stratify=y)

In [ ]:
nusvc_model = nusvc(x_train, y_train, x_validation, y_validation)
test_model(nusvc_model,x_validation,y_validation)
cross_val(nusvc_model, x_train, y_train)

In [ ]:
best_features = best_features(nusvc_model,X_train).to_list()
if "Opportunity_ID" not in best_features: 
    best_featu.append("Opportunity_ID")

In [ ]:
y_pred = nusvc_model.predict_proba(X_test)[:,1]
submission_nusvc = pd.DataFrame(data={'Opportunity_ID':X_test['Opportunity_ID'], 'Target': y_pred})
submission_nusvc = submission_nusvc.groupby("Opportunity_ID").agg({"Target":"mean"}).reset_index()
submission_nusvc.to_csv('../submits/nusvc_cat_boost_encoding.csv', index=False)

In [ ]:
X_train_best_features_2 = X_train.loc[:,best_features]
X_test_best_features_2 = X_test.loc[:,best_features]

In [ ]:
x_best_train, x_best_validation, y_best_train, y_best_validation = train_test_split(X_train_best_features, y, test_size=0.3, stratify=y)

In [ ]:
nusvc_model_2 = nusvc(x_best_train, y_best_train, x_best_validation, y_best_validation)
test_model(nusvc_model_2,x_best_validation,y_best_validation)
cross_val(nusvc_model_2, x_best_train, y_best_train)

In [ ]:
y_pred_2 = nusvc_model_2.predict_proba(X_test_best_features)[:,1]
submission_nusvc_2 = pd.DataFrame(data={'Opportunity_ID':X_test_best_features['Opportunity_ID'], 'Target': y_pred_2})
submission_nusvc_2 = submission_nusvc_2.groupby("Opportunity_ID").agg({"Target":"mean"}).reset_index()
submission_nusvc_2.to_csv('../submits/nusvc_2_best_features_with_cat_boost_encoding.csv', index=False)